In [28]:
from bs4 import BeautifulSoup
import requests as requests
import time

**I have to insert a try except because some posts do not have a title**

In [43]:
titles = []

start = time.time()

for i in range(1,2):
    
    try:
        url_posts = 'https://stacker.news/items/%d' % (i)
        response = requests.get(url_posts)
        soup = BeautifulSoup(response.text, 'html.parser')
    
        title = soup.find_all('a', class_='item_title__FH7AS text-reset me-2')
        
        for e in title:
            titles.append(e.get_text())
        
    except:
        continue

end = time.time()

In [44]:
print("The time of execution of above program is :",
      (end-start), "s")

The time of execution of above program is : 0.7928695678710938 s


In [45]:
titles

['El Salvador Makes Bitcoin Legal Tender']